In [1]:
from helpers.charting_tools import Charting
from helpers.data_processing import add_ti
from BookWorm import BookWorm, BinanceWrapper
from PIL import Image

In [2]:
worm = BookWorm()

In [32]:
candles = worm.historical_candles(start_time='January 1 2018', end_time='February 1 2019', api_wrapper=BinanceWrapper('5lJ0uGit9PuUxHka3hBWhPmsi7dWyxEwvEntUZFKmm0xfNz3VjHWi5WSr5W1VBJV',
                                                      'BFWVs8ko7Cd4sjdQ9amGJTnToGWy9TbQWIjeorSCj23FGiwFaknzkgLPcrgWrxsw'), 
                  symbol='ETHBTC', interval='1m')

In [33]:
candles = add_ti(candles)

In [34]:
def split_candles(df, num_rows=30, step=10):
    """Split a DataFrame of candlestick data into a list of smaller DataFrames each with num_rows rows"""
    
    slices = []
    
    for row_i in range(0, df.shape[0] - num_rows, step):
        small_df = df.iloc[row_i:row_i+num_rows, :]
        slices.append(small_df)
        
    return slices

In [35]:
def price_returns(df, num_rows=30, num_into_fut=5, step=10):
    labels = []
    
    for row_i in range(0, df.shape[0] - num_rows - num_into_fut, step):
        # skip all iterations while row_i < num_rows since nothing yet to create a label for
        if row_i <= num_rows: continue
        
        vf, vi = df['close'][row_i+num_into_fut], df['close'][row_i]
        price_return = (vf - vi) / vi
        labels.append(price_return)
    return labels

In [36]:
# split candles into 30 period and a label
candles_sliced = split_candles(candles)
labels_candles_sliced = price_returns(candles)
# we need to remove candle slices without a label from candles_sliced
candles_sliced = candles_sliced[len(candles_sliced)-len(labels_candles_sliced):]

assert len(candles_sliced) == len(labels_candles_sliced)
len(candles_sliced)

56677

In [37]:
def create_charts(candles_sliced, save_path):
    """Create a chart image for each in sliced_candles and return a list of paths to those images"""
    from tqdm import tqdm_notebook as tqdm
    import warnings
    warnings.filterwarnings("ignore")
    
    i = 0
    paths_to_images = []
    for small_df in tqdm(candles_sliced):
        chart = Charting(small_df, 'time', 'close')
        
        path = save_path + 'chart_{}.png'.format(i)
        chart.chart_to_image(path)
        paths_to_images.append(path)
        i += 1
    return paths_to_images        

In [9]:
paths_to_images = create_charts(candles_sliced, "images/")

KeyboardInterrupt: 

In [ ]:
paths_to_images = [ 'images/chart_{}.png'.format(i) for i in range(len(candles_sliced)) ]

In [ ]:
Image.open(paths_to_images[100])

In [38]:
def normalize_series(ser):
    return (ser-ser.min())/(ser.max()-ser.min())

In [39]:
try: candles = candles.drop('time', axis=1)
except: pass
candles.shape

(566841, 11)

In [40]:
candles.apply(normalize_series, axis=0)

open      high       low     close    volume     sma20      macd  \
24      0.294695  0.294816  0.295438  0.294691  0.007892  0.297465  1.000000   
25      0.294988  0.294634  0.294983  0.294267  0.010340  0.297578  0.957259   
26      0.294766  0.294846  0.293143  0.294024  0.016392  0.297686  0.917540   
27      0.293887  0.293807  0.292859  0.293479  0.007877  0.297774  0.880663   
28      0.293281  0.293565  0.293355  0.293479  0.010889  0.297817  0.846456   
...          ...       ...       ...       ...       ...       ...       ...   
566860  0.063916  0.063924  0.064307  0.064038  0.002764  0.064186  0.578027   
566861  0.064057  0.063924  0.064388  0.063998  0.004898  0.064187  0.578029   
566862  0.064017  0.063934  0.064317  0.063957  0.004241  0.064180  0.578031   
566863  0.064077  0.063975  0.064317  0.063978  0.004918  0.064176  0.578032   
566864  0.064098  0.064096  0.064337  0.064069  0.011441  0.064166  0.578032   

             obv  bb20_low  bb20_mid   bb20_up  
24      0.724619  0.298127  0.297465  0.294494  
25      0.724342  0.298357  0.297578  0.294490  
26      0.723903  0.298665  0.297686  0.294401  
27      0.723693  0.298952  0.297774  0.294294  
28      0.723693  0.299108  0.297817  0.294225  
...          ...       ...       ...       ...  
566860  0.008281  0.066320  0.064186  0.062701  
566861  0.008150  0.066320  0.064187  0.062701  
566862  0.008037  0.066322  0.064180  0.062685  
566863  0.008168  0.066319  0.064176  0.062680  
566864  0.008475  0.066325  0.064166  0.062655  

[566841 rows x 11 columns]

In [41]:
import torch
torch.backends.cudnn.benchmark = True

# Parameters
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 5}

In [42]:
def _train(train_gen, model, optim, error_func):
    losses = []
    
    for batch, labels in train_gen:
        batch, labels = batch.cuda().float(), labels.cuda().float()
        # set model to train mode
        model.train()
        
        # clear gradients
        model.zero_grad()
        
        output = model(batch)
        loss = error_func(output, labels)
        loss.backward()
        optim.step()
        
        
        losses.append(loss)
        
    return round(float(sum(losses) / len(losses)), 6)

In [43]:
def _valid(valid_gen, model, optim, error_func):
    with torch.set_grad_enabled(False):
        losses = []

        for batch, labels in valid_gen:
            batch, labels = batch.cuda().float(), labels.cuda().float()
            
            # set to eval mode
            model.eval()
            
            # clear gradients
            model.zero_grad()

            output = model(batch)
            loss = error_func(output, labels)

            losses.append(loss)
        
    return round(float(sum(losses) / len(losses)), 6)

In [44]:
def _test(test_gen, model, optim, error_func):
    with torch.set_grad_enabled(False):
        losses = []

        for batch, labels in valid_gen:
            batch, labels = batch.cuda().float(), labels.cuda().float()
            
            # set to eval mode
            model.eval()
            
            # clear gradients
            model.zero_grad()

            output = model(batch)
            loss = error_func(output, labels)

            losses.append(loss)
        
    return round(float(sum(losses) / len(losses)), 6)

In [45]:
def train(model, model_name, optim, num_epochs, train_gen, valid_gen, test_gen=None):
    """Train a PyTorch model with optim as optimizer strategy"""
    
    for epoch_i in range(num_epochs):
        
        
        def RMSE(x, y):
            
            # have to squish x into a rank 1 tensor with batch_size length with the outputs we want
            if model_name == 'resnet':
                 # torch.Size([64, 1])
                x = x.squeeze(1)
            elif model_name == 'gru':
                # torch.Size([64, 30, 1])
                x = x[:, 29, :] # take only the last prediction from the 30 time periods in our matrix
                x = x.squeeze(1)
            mse = torch.nn.MSELoss()
            return torch.sqrt(mse(x, y))
        
        
        # forward and backward passes of all batches inside train_gen
        train_loss = _train(train_gen, model, optim, RMSE)
        valid_loss = _valid(valid_gen, model, optim, RMSE)
        
        # run on test set if provided
        if test_gen: test_output = _test(test_gen, model, optim)
        else: test_output = "no test selected"
        print("train loss: {}, valid loss: {}, test output: {}".format(train_loss, valid_loss, test_output))

In [ ]:
from models.CNN.CNN import CNN
cnn = CNN().cuda().float()

In [ ]:
from helpers.datasets import DFTimeSeriesDataset, ChartImageDataset
from torch.utils.data import *
# create dataloaders
# specify the split between train_df and valid_df from the process of splitting dataset_windows 
split = 0.7

s = int(len(candles_sliced) * 0.7)
while s % params['batch_size'] != 0:
    s += 1

# create two ChartImageDatasets, split by split, for the purpose of creating a DataLoader for the specific model
train_ds_cnn = ChartImageDataset(paths_to_images[:s], labels_candles_sliced[:s])
valid_ds_cnn = ChartImageDataset(paths_to_images[s:], labels_candles_sliced[s:])
train_gen_cnn = DataLoader(train_ds_cnn, **params)
valid_gen_cnn = DataLoader(valid_ds_cnn, **params)

In [ ]:
train(cnn, 'resnet', torch.optim.Adam(cnn.parameters(), 1e-3), 15, train_gen_cnn, valid_gen_cnn)

In [49]:
from models.GRU.GRU import GRUnet
gru = GRUnet(num_features=12, num_rows=30, batch_size=64, hidden_size=500, num_layers=5).cuda().float()

In [50]:
from helpers.datasets import DFTimeSeriesDataset, ChartImageDataset
from torch.utils.data import *
# create dataloaders
# specify the split between train_df and valid_df from the process of splitting dataset_windows 
split = 0.7

s = int(len(candles_sliced) * 0.7)
while s % params['batch_size'] != 0:
    s += 1

# create two ChartImageDatasets, split by split, for the purpose of creating a DataLoader for the specific model
train_ds_gru = DFTimeSeriesDataset(candles_sliced[:s], labels_candles_sliced[:s])
valid_ds_gru = DFTimeSeriesDataset(candles_sliced[s:], labels_candles_sliced[s:])
train_gen_gru = DataLoader(train_ds_gru, **params, drop_last=True)
valid_gen_gru = DataLoader(valid_ds_gru, **params, drop_last=True)

In [51]:
train(gru, 'gru', torch.optim.Adam(gru.parameters(), 1e-3), 5, train_gen_gru, valid_gen_gru)

train loss: 0.003663, valid loss: 0.001888, test output: no test selected
train loss: 0.0021, valid loss: 0.002282, test output: no test selected
train loss: 0.002023, valid loss: 0.001827, test output: no test selected
train loss: 0.002007, valid loss: 0.001728, test output: no test selected
train loss: 0.001978, valid loss: 0.001733, test output: no test selected


In [52]:
from helpers.saving_models import save_model, load_model

save_model(gru, 'gru_weights')

In [53]:
from BackTest import BackTest

ImportError: attempted relative import with no known parent package